In [1]:
import os.path
import re
import sys

import ipysheet
from clingo.script import register_script

from util.run import explain

sys.path.append('..')
from Implementation.apia_control_loop import *

In [2]:
def generate_sheet(rows=5, columns=5):
    sheet = ipysheet.sheet(rows=rows, columns=columns, column_width=[20] + [5 for _ in range(1, columns)])
    for row in range(rows):
        for column in range(columns):
            ipysheet.cell(row, column, "")
    return sheet


In [3]:
sheet = generate_sheet(rows=13, columns=9)

ipysheet.cell(0, 0, "happened(wait, T)")
ipysheet.cell(1, 0, 'happened(select(policy_compliant(greeted_by("Bob", "Alice"))), T)')
ipysheet.cell(2, 0, 'happened(start(1), T)')
ipysheet.cell(3, 0, 'happened(stop(1), T)')
ipysheet.cell(4, 0, 'happened(move_through("Alice", "d12"), T)')
ipysheet.cell(5, 0, 'happened(move_through("Alice", "d23"), T)')
ipysheet.cell(6, 0, 'happened(move_through("Alice", "d34"), T)')
ipysheet.cell(7, 0, 'happened(greet("Alice", "Bob"), T)')
ipysheet.cell(8, 0, 'observed(in_room("Alice", "r2"), T)')
ipysheet.cell(9, 0, 'observed(in_room("Alice", "r3"), T)')
ipysheet.cell(10, 0, 'observed(in_room("Alice", "r4"), T)')
ipysheet.cell(11, 0, 'observed(greeted_by("Bob", "Alice"), T)')
ipysheet.cell(12, 0, '-observed(greeted_by("Bob", "Alice"), T)')

ipysheet.cell(0, 1, "X")
ipysheet.cell(0, 8, "X")
ipysheet.cell(1, 1, "X")
ipysheet.cell(2, 2, "X")
ipysheet.cell(3, 7, "X")
ipysheet.cell(4, 3, "X")
ipysheet.cell(5, 4, "X")
ipysheet.cell(6, 5, "X")
ipysheet.cell(7, 6, "X")

ipysheet.cell(8, 4, "X")
ipysheet.cell(9, 5, "X")
ipysheet.cell(10, 6, "X")

ipysheet.cell(11, 7, "X")
ipysheet.cell(12, 2, "X")
ipysheet.cell(12, 3, "X")
ipysheet.cell(12, 4, "X")
ipysheet.cell(12, 5, "X")
ipysheet.cell(12, 6, "X")

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value=''), Cell(column_en…

In [4]:
df = ipysheet.pandas_loader.to_dataframe(sheet)
df

,A,B,C,D,E,F,G,H,I
0,"happened(wait, T)",X,,,,,,,X
1,"happened(select(policy_compliant(greeted_by(""B...",X,,,,,,,
2,"happened(start(1), T)",,X,,,,,,
3,"happened(stop(1), T)",,,,,,,X,
4,"happened(move_through(""Alice"", ""d12""), T)",,,X,,,,,
5,"happened(move_through(""Alice"", ""d23""), T)",,,,X,,,,
6,"happened(move_through(""Alice"", ""d34""), T)",,,,,X,,,
7,"happened(greet(""Alice"", ""Bob""), T)",,,,,,X,,
8,"observed(in_room(""Alice"", ""r2""), T)",,,,X,,,,
9,"observed(in_room(""Alice"", ""r3""), T)",,,,,X,,,


In [5]:
fact_dict = {}

for rindex, row in df.iterrows():
    fact = None
    for cindex, cell in enumerate(row):
        if cindex == 0:
            fact = cell
            fact_dict[cell] = set()
        elif cell == "X":
            fact_dict[fact].add(cindex - 1)

fact_dict

{'happened(wait, T)': {0, 7},
 'happened(select(policy_compliant(greeted_by("Bob", "Alice"))), T)': {0},
 'happened(start(1), T)': {1},
 'happened(stop(1), T)': {6},
 'happened(move_through("Alice", "d12"), T)': {2},
 'happened(move_through("Alice", "d23"), T)': {3},
 'happened(move_through("Alice", "d34"), T)': {4},
 'happened(greet("Alice", "Bob"), T)': {5},
 'observed(in_room("Alice", "r2"), T)': {3},
 'observed(in_room("Alice", "r3"), T)': {4},
 'observed(in_room("Alice", "r4"), T)': {5},
 'observed(greeted_by("Bob", "Alice"), T)': {6},
 '-observed(greeted_by("Bob", "Alice"), T)': {1, 2, 3, 4, 5}}

In [6]:
action_string = "^(-?)(happened|observed)\(\s*(.*)\s*,\s*(T)\s*\)$"
action_pattern = re.compile(action_string)


def fill_time(fact, time):
    if action_pattern.match(fact):
        return action_pattern.sub(r"\1\2(\3, {})".format(time), fact)
    return fact



In [7]:
program_dict = dict()

for fact in fact_dict:  # type: str
    for time in fact_dict[fact]:
        if fact.startswith("happened") or fact.startswith("-happened"):
            key = "#program observations_{}".format(time + 1)
            if key not in program_dict:
                program_dict[key] = set()
            program_dict[key].add(fill_time(fact, time))
        elif fact.startswith("observed") or fact.startswith("-observed"):
            key = "#program observations_{}".format(time)
            if key not in program_dict:
                program_dict[key] = set()
            program_dict[key].add(fill_time(fact, time))

program_dict

{'#program observations_1': {'-observed(greeted_by("Bob", "Alice"), 1)',
  'happened(select(policy_compliant(greeted_by("Bob", "Alice"))), 0)',
  'happened(wait, 0)'},
 '#program observations_8': {'happened(wait, 7)'},
 '#program observations_2': {'-observed(greeted_by("Bob", "Alice"), 2)',
  'happened(start(1), 1)'},
 '#program observations_7': {'happened(stop(1), 6)'},
 '#program observations_3': {'-observed(greeted_by("Bob", "Alice"), 3)',
  'happened(move_through("Alice", "d12"), 2)',
  'observed(in_room("Alice", "r2"), 3)'},
 '#program observations_4': {'-observed(greeted_by("Bob", "Alice"), 4)',
  'happened(move_through("Alice", "d23"), 3)',
  'observed(in_room("Alice", "r3"), 4)'},
 '#program observations_5': {'-observed(greeted_by("Bob", "Alice"), 5)',
  'happened(move_through("Alice", "d34"), 4)',
  'observed(in_room("Alice", "r4"), 5)'},
 '#program observations_6': {'happened(greet("Alice", "Bob"), 5)',
  'observed(greeted_by("Bob", "Alice"), 6)'}}

In [8]:
with open("run_observations.lp", "w") as lp_file:
    for program, facts in program_dict.items():
        lp_file.write(program)
        lp_file.write('.\n')
        lp_file.writelines("{}.\n".format(fact) for fact in program_dict[program])

In [9]:
script_dir = os.path.join('..', 'Implementation')

In [10]:
# Change these if you want to run for another example:
instance = os.path.join('..', 'Examples', 'Authorization', 'Example_A', 'instance.lp')
domain_encoding = os.path.join('..', 'Examples', 'Authorization', 'Example_A', 'domain_encoding.lp')

max_timestep = 7

In [11]:
# Change how the scripts are evaluated
debug = False  # Turn on additional debug info
configuration = APIAConfiguration(authorization=APIAAuthorizationSetting.PARANOID,
                                  obligation=APIAObligationSetting.SUBORDINATE)


In [12]:
clingo_files = get_clingo_files(script_dir=Path(script_dir), files=(instance, domain_encoding, "run_observations.lp"), debug=debug)
clingo_files_no_debug = get_clingo_files(script_dir=Path(script_dir), files=(instance, domain_encoding, "run_observations.lp"), debug=False, include_script_defs=True)
print(clingo_files)
clingo_args = (
    '--opt-mode=optN',
    '--parallel-mode', f'{1}',
    '--warn=no-atom-undefined',
    '2' if debug else '1',
)

history: deque[clingo.Symbol] = deque()
observation_subprograms: deque[ASPSubprogramInstantiation] = deque()

(PosixPath('../Implementation/aaa_axioms.lp'), PosixPath('../Implementation/aia_theory_of_intentions.lp'), PosixPath('../Implementation/aia_history_rules.lp'), PosixPath('../Implementation/aia_intended_action_rules.lp'), PosixPath('../Implementation/aopl_authorization_compliance.lp'), PosixPath('../Implementation/aopl_obligation_compliance.lp'), PosixPath('../Implementation/general_axioms.lp'), PosixPath('../Implementation/apia_cr_prolog.lp'), PosixPath('../Implementation/apia_policy.lp'), PosixPath('../Implementation/apia_compliance_check.lp'), PosixPath('../Implementation/apia_optimization_priorities.lp'), PosixPath('../Implementation/apia_show.lp'), PosixPath('../Implementation/apia_patch.lp'), PosixPath('../Examples/Authorization/Example_A/instance.lp'), PosixPath('../Examples/Authorization/Example_A/domain_encoding.lp'), PosixPath('run_observations.lp'))


In [13]:
current_timestep = 0

In [60]:
print('Timestep {}/{}'.format(current_timestep,max_timestep))

Timestep 7/7


In [61]:
print('  Step 1: Interpret observations')
step_2_unobserved_actions_count, step_1_answer_set = interpret_observations(clingo_files=clingo_files,
                                                                       clingo_args=clingo_args,
                                                                       history=history,
                                                                       observation_subprograms=observation_subprograms,
                                                                       current_timestep=current_timestep,
                                                                       max_timestep=max_timestep,
                                                                       configuration=configuration,
                                                                       debug=debug)

  Step 1: Interpret observations
    Grounding...
    Solving...
    Unobserved actions: 0


In [62]:
# does not work yet:
# explain(clingo_files, answer_set=step_1_answer_set, clingo_args=clingo_args, grounding_context=GroundingContext)

In [63]:
# Step 2: Find intended action
print()
print('  Step 2: Find intended action')
step_3_intended_actions, step_2_answer_set = find_intended_action(clingo_files=clingo_files,
                                                             clingo_args=clingo_args,
                                                             history=history,
                                                             step_2_unobserved_actions_count=step_2_unobserved_actions_count,
                                                             observation_subprograms=observation_subprograms,
                                                             current_timestep=current_timestep,
                                                             max_timestep=max_timestep,
                                                             configuration=configuration,
                                                             debug=debug)


  Step 2: Find intended action
    Grounding...
    Solving...
    Intended action: wait


In [64]:

# Step 3: Perform intended action
print()
print('  Step 3: Do intended action')
do_intended_action(step_3_intended_actions=step_3_intended_actions,
                   history=history,
                   current_timestep=current_timestep)


  Step 3: Do intended action
    Doing wait


In [65]:

# Step 4: Observe world
print()
print('  Step 4: Observe world')
observe_world(current_timestep=current_timestep,
              observation_subprograms=observation_subprograms)


  Step 4: Observe world
    Getting observations from #program observations_8.


In [66]:
current_timestep += 1